In [1]:
#!/usr/bin/env python

import numpy as np
import pandas as pd
from teamName import getMyPosition as getPosition

nInst = 0
nt = 0
commRate = 0.0010
dlrPosLimit = 10000

def loadPrices(fn):
    global nt, nInst
    #df=pd.read_csv(fn, sep='\s+', names=cols, header=None, index_col=0)
    df=pd.read_csv(fn, sep='\s+', header=None, index_col=None)
    nt, nInst = df.values.shape
    return (df.values).T

pricesFile="./data/prices.txt"
prcAll = loadPrices(pricesFile)
print ("Loaded %d instruments for %d days" % (nInst, nt))

currentPos = np.zeros(nInst)

pred_df = pd.read_csv('pred.csv')


def calcPL(prcHist):
    cash = 0
    curPos = np.zeros(nInst)
    totDVolume = 0
    totDVolumeSignal = 0
    totDVolumeRandom = 0
    value = 0
    todayPLL = []
    (_,nt) = prcHist.shape
    for t in range(1+250,251+250): 
        prcHistSoFar = prcHist[:,:t]
        newPosOrig = getPosition(pred_df, t-1-250)
        curPrices = prcHistSoFar[:,-1] #prcHist[:,t-1]
        posLimits = np.array([int(x) for x in dlrPosLimit / curPrices])
        clipPos = np.clip(newPosOrig, -posLimits, posLimits)
        newPos = np.array([np.trunc(x) for x in clipPos])
        deltaPos = newPos - curPos
        dvolumes = curPrices * np.abs(deltaPos)
        dvolume = np.sum(dvolumes)
        totDVolume += dvolume
        comm = dvolume * commRate
        cash -= curPrices.dot(deltaPos) + comm
        curPos = np.array(newPos)
        posValue = curPos.dot(curPrices)
        todayPL = cash + posValue - value
        todayPLL.append(todayPL)
        value = cash + posValue
        ret = 0.0
        if (totDVolume > 0):
            ret = value / totDVolume
        print ("Day %d value: %.2lf todayPL: $%.2lf $-traded: %.0lf return: %.5lf" % (t,value, todayPL, totDVolume, ret))
    pll = np.array(todayPLL)
    (plmu,plstd) = (np.mean(pll), np.std(pll))
    annSharpe = 0.0
    if (plstd > 0):
        annSharpe = np.sqrt(250) * plmu / plstd
    return (plmu, ret, plstd, annSharpe, totDVolume)



(meanpl, ret, plstd, sharpe, dvol) = calcPL(prcAll)
score = meanpl - 0.1*plstd
print ("=====")
print ("mean(PL): %.1lf" % meanpl)
print ("return: %.5lf" % ret)
print ("StdDev(PL): %.2lf" % plstd)
print ("annSharpe(PL): %.2lf " % sharpe)
print ("totDvolume: %.0lf " % dvol)
print ("Score: %.2lf" % score)




Loaded 50 instruments for 500 days
Day 251 value: -9.97 todayPL: $-9.97 $-traded: 9967 return: -0.00100
Day 252 value: 24.80 todayPL: $34.77 $-traded: 10002 return: 0.00248
Day 253 value: 21.89 todayPL: $-2.91 $-traded: 10002 return: 0.00219
Day 254 value: -24.29 todayPL: $-46.18 $-traded: 29993 return: -0.00081
Day 255 value: -90.79 todayPL: $-66.49 $-traded: 49926 return: -0.00182
Day 256 value: -75.68 todayPL: $15.11 $-traded: 69857 return: -0.00108
Day 257 value: -95.64 todayPL: $-19.97 $-traded: 89822 return: -0.00106
Day 258 value: -163.10 todayPL: $-67.46 $-traded: 89891 return: -0.00181
Day 259 value: -183.06 todayPL: $-19.96 $-traded: 109853 return: -0.00167
Day 260 value: -273.38 todayPL: $-90.31 $-traded: 109955 return: -0.00249
Day 261 value: -329.27 todayPL: $-55.89 $-traded: 109989 return: -0.00299
Day 262 value: -406.07 todayPL: $-76.80 $-traded: 110090 return: -0.00369
Day 263 value: -429.03 todayPL: $-22.96 $-traded: 130073 return: -0.00330
Day 264 value: -401.28 today